In [ ]:
import os
import keras
from keras.layers import *
import numpy as np
from sklearn.model_selection import train_test_split
from keras import backend as K
from sklearn.metrics import classification_report
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import sklearn
import torch
import random
import math
from matplotlib import pyplot as plt

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
def prepare_data(arr1,arr2,arr3,size):
    arr_1_n=[]
    arr_2_n=[]
    arr_3_n=[]
    for i in range(len(arr1)):
        arr_1_n.append(np.array(divide_and_split_data(arr1[i],size),dtype=object))
        arr_2_n.append(np.array(divide_and_split_data(arr2[i],size),dtype=object))
        arr_3_n.append(np.array(divide_and_split_data(arr3[i],size),dtype=object))
    return np.array(arr_1_n,dtype=object),np.array(arr_2_n,dtype=object),np.array(arr_3_n,dtype=object)

In [ ]:
def merge_arr(arr):
    arr_n=[]
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            arr_n.append(arr[i][j])
    return np.array(arr_n,dtype=object)

In [ ]:
def filter_final_data(t_f_1,s_f_1,c_1,t_f_2,s_f_2,c_2):
    l1=len(t_f_1)
    l2=len(t_f_2)
    if l1<l2:
        return t_f_1,s_f_1,c_1,t_f_2[:l1],s_f_2[:l1],c_2[:l1]
    elif l1>l2:
        return t_f_1[:l2],s_f_1[:l2],c_1[:l2],t_f_2,s_f_2,c_2
    else:
        return t_f_1,s_f_1,c_1,t_f_2,s_f_2,c_2

In [ ]:
def normalize(arr):
    s=[]
    for i in arr:
        s.append(np.amax(i))
    return arr/max(s)

In [ ]:
def stack_features_and_output(t_f_1,s_f_1,t_f_2,s_f_2,c_1,c_2):
    x=[]
    y=[]
    p=len(t_f_1)
    q=len(t_f_2)
    if q<p:
        for i in range(q):
            x.append(np.vstack((t_f_1[i],s_f_1[i],t_f_2[i],s_f_2[i])))
            y.append(np.vstack((c_1[i],c_2[i])))
    else:
        for i in range(p):
            x.append(np.vstack((t_f_1[i],s_f_1[i],t_f_2[i],s_f_2[i])))
            y.append(np.vstack((c_1[i],c_2[i])))
        
    return np.array(x,dtype=object),np.array(y,dtype=object)

In [ ]:
def divide_and_split_data(arr,size):
    if len(arr)==size:
        return arr
    val=len(arr)%size
    pad=size-val
    arr_n=np.append(arr,[0]*pad)
    return np.split(arr_n, len(arr_n)/size)

In [ ]:
def generate_X_and_Y(t_feature_1_padded,s_feature_1_padded,feature_1_class,t_feature_2_padded,s_feature_2_padded,feature_2_class,size):
    t_f_1,s_f_1,c_1=prepare_data(t_feature_1_padded,s_feature_1_padded,feature_1_class,size)
    t_f_2,s_f_2,c_2=prepare_data(t_feature_2_padded,s_feature_2_padded,feature_2_class,size)

    t_f_1,s_f_1,c_1=merge_arr(t_f_1),merge_arr(s_f_1),merge_arr(c_1)
    t_f_2,s_f_2,c_2=merge_arr(t_f_2),merge_arr(s_f_2),merge_arr(c_2)
    


    t_f_1,s_f_1,c_1,t_f_2,s_f_2,c_2=filter_final_data(t_f_1,s_f_1,c_1,t_f_2,s_f_2,c_2)
    
    
    p=c_1
    q=c_2
    t_f_1,s_f_1,t_f_2,s_f_2=normalize(t_f_1),normalize(s_f_1),normalize(t_f_2),normalize(s_f_2)

    X,Y=stack_features_and_output(t_f_1,s_f_1,t_f_2,s_f_2,c_1,c_2)

    X = np.asarray(X).astype(np.float32)
    Y = np.asarray(Y).astype(np.float32)
    X=X.transpose((0,2,1))
    Y=Y.transpose((0,2,1))
    
    return X,Y,p,q


In [ ]:
def class_count(arr):
    a1=[]
    for i in arr:
        
        a1.append(sum(i==1))
    return sum(a1)

In [ ]:
def class_zero_one_count(arr):
    a1=[]
    a0=[]
    for i in arr:
        a1.append(sum(i==1))
        a0.append(sum(i==0))
    return sum(a1),sum(a0)
    

In [ ]:
def class_weights(c1,c2):
    return [c1/(c1+c2),c2/(c1+c2)]
    

In [ ]:
def class_avg_weight(x,y,N):
    return [x/N,y/N]

In [ ]:
def get_random_idx(arr,size):
    samples=math.floor(len(arr)*size)
    idx=random.sample(range(0, len(arr)), samples)
    return idx 

In [ ]:
def split_test_train(arr,idx):
    train=[]
    test=[]
    for i in range(len(arr)):
        if i in idx:
            test.append(arr[i])
        else:
            train.append(arr[i])
    return np.array(train,dtype=object),np.array(test,dtype=object)

In [ ]:
def split_test_train2(arr,idx):
    size=0.10
    samples=math.floor(len(arr)*size)
    x=len(arr)-samples
    return np.array(arr[:x],dtype=object),np.array(arr[x+1:],dtype=object)

In [ ]:
def load_model(path):
    model = keras.models.load_model(path)
    return model

In [ ]:
t_1_001=np.load("Data/001/features/t_1_001.npy",allow_pickle=True)
s_1_001=np.load("Data/001/features/s_1_001.npy",allow_pickle=True)
c_1_001=np.load("Data/001/features/c_1_001.npy",allow_pickle=True)
t_2_001=np.load("Data/001/features/t_2_001.npy",allow_pickle=True)
s_2_001=np.load("Data/001/features/s_2_001.npy",allow_pickle=True)
c_2_001=np.load("Data/001/features/c_2_001.npy",allow_pickle=True)

In [ ]:
t_1_002=np.load("Data/002/features/t_1_002.npy",allow_pickle=True)
s_1_002=np.load("Data/002/features/s_1_002.npy",allow_pickle=True)
c_1_002=np.load("Data/002/features/c_1_002.npy",allow_pickle=True)
t_2_002=np.load("Data/002/features/t_2_002.npy",allow_pickle=True)
s_2_002=np.load("Data/002/features/s_2_002.npy",allow_pickle=True)
c_2_002=np.load("Data/002/features/c_2_002.npy",allow_pickle=True)

In [ ]:
idx_001=get_random_idx(t_1_001,0.10)
idx_002=get_random_idx(t_1_002,0.10)

In [ ]:
t_1_001_train,t_1_001_test=split_test_train2(t_1_001,idx_001)
s_1_001_train,s_1_001_test=split_test_train2(s_1_001,idx_001)
c_1_001_train,c_1_001_test=split_test_train2(c_1_001,idx_001)
t_2_001_train,t_2_001_test=split_test_train2(t_2_001,idx_001)
s_2_001_train,s_2_001_test=split_test_train2(s_2_001,idx_001)
c_2_001_train,c_2_001_test=split_test_train2(c_2_001,idx_001)

In [ ]:
t_1_002_train,t_1_002_test=split_test_train2(t_1_002,idx_002)
s_1_002_train,s_1_002_test=split_test_train2(s_1_002,idx_002)
c_1_002_train,c_1_002_test=split_test_train2(c_1_002,idx_002)
t_2_002_train,t_2_002_test=split_test_train2(t_2_002,idx_002)
s_2_002_train,s_2_002_test=split_test_train2(s_2_002,idx_002)
c_2_002_train,c_2_002_test=split_test_train2(c_2_002,idx_002)

In [ ]:
size=200

In [ ]:
X_001_train,Y_001_train,c1_train,c2_train=generate_X_and_Y(t_1_001_train,s_1_001_train,c_1_001_train,t_2_001_train,s_2_001_train,c_2_001_train,size)
print("Teacher - 001")
print("Train-")
print(X_001_train.shape)
print(Y_001_train.shape)
X_001_test,Y_001_test,c1_test,c2_test=generate_X_and_Y(t_1_001_test,s_1_001_test,c_1_001_test,t_2_001_test,s_2_001_test,c_2_001_test,size)
print("Test-")
print(X_001_test.shape)
print(Y_001_test.shape)

In [ ]:
X_002_train,Y_002_train,p1_train,p2_train=generate_X_and_Y(t_1_002_train,s_1_002_train,c_1_002_train,t_2_002_train,s_2_002_train,c_2_002_train,size)
print("Teacher - 002")
print("Train-")
print(X_002_train.shape)
print(Y_002_train.shape)
X_002_test,Y_002_test,p1_test,p2_test=generate_X_and_Y(t_1_002_test,s_1_002_test,c_1_002_test,t_2_002_test,s_2_002_test,c_2_002_test,size)
print("Test-")
print(X_002_test.shape)
print(Y_002_test.shape)

In [ ]:
# x_train,y_train=X_001_train,Y_001_train
# x_test,y_test=X_001_test,Y_001_test
# s1=class_count(c1_train)
# s2=class_count(c2_train)
# t_id="001"

x_train,y_train=X_002_train,Y_002_train
x_test,y_test=X_002_test,Y_002_test
s1=class_count(p1_train)
s2=class_count(p2_train)
t_id="002"

# x_train,y_train=np.concatenate((X_001_train,X_002_train),axis=0),np.concatenate((Y_001_train,Y_002_train))
# x_test,y_test=np.concatenate((X_001_test,X_002_test),axis=0),np.concatenate((Y_001_test,Y_002_test))
# q1,q2=np.concatenate((c1_train,p1_train)),np.concatenate((c2_train,p2_train))
# s1=class_count(q1)
# s2=class_count(q2)
# np.save("x_train_002.npy",x_train)
# np.save("y_train_002.npy",y_train)
# np.save("x_test_002.npy",x_test)
# np.save("y_test_002.npy",y_test)

In [ ]:
cw=class_weights(s1,s2)
cw

In [ ]:
print(x_train.shape) 
print(y_train.shape)

In [ ]:
model = keras.models.Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(size,4),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=16, kernel_size=3, activation='relu',padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Conv1D(filters=2, kernel_size=3, activation='sigmoid',padding='same',use_bias=True))
#opt = keras.optimizers.Adam(learning_rate=0.0001)
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train,y_train,epochs=50,verbose=1,class_weight=cw,batch_size=64)

In [ ]:
r=model.evaluate(x_test,y_test,batch_size=64)
print(r)

In [ ]:
y_pred=torch.Tensor(model.predict(x_test))
y_true=torch.Tensor(y_test)
threshold=0.5
for i in range(y_pred.shape[0]):
    y0 = y_pred[i,:,0]
    y1 = y_pred[i,:,1]
    y_pred[i,:,0] = (y0>threshold).float()
    y_pred[i,:,1] = (y1>threshold).float()
y_pred.shape
y0 = y_pred[:,:,0]
y1 = y_pred[:,:,1]
y0 = y0.reshape(-1)
y1 = y1.reshape(-1)

yA = y_true[:,:,0]
yB = y_true[:,:,1]
yA = yA.reshape(-1)
yB = yB.reshape(-1)

In [ ]:
print("Accuracy-",sklearn.metrics.accuracy_score(yA,y0))
print("F1 Score-",sklearn.metrics.f1_score(yA,y0))

In [ ]:
print("Accuracy-",sklearn.metrics.accuracy_score(yB,y1))
print("F1 Score-",sklearn.metrics.f1_score(yB,y1))

In [ ]:
model.save("models/model_"+ t_id +".h5")

In [ ]:
model=load_model("models/model_"+ t_id +".h5")

In [ ]:
x_test.shape

In [ ]:
y_pred=model.predict(x_test)
y_true=torch.Tensor(y_test)
threshold=0.5
for i in range(y_pred.shape[0]):
    y0 = y_pred[i,:,0]
    y1 = y_pred[i,:,1]
    y_pred[i,:,0] = (y0>threshold).float()
    y_pred[i,:,1] = (y1>threshold).float()
y_pred.shape
y0 = y_pred[:,:,0]
y1 = y_pred[:,:,1]
y0 = y0.reshape(-1)
y1 = y1.reshape(-1)

yA = y_true[:,:,0]
yB = y_true[:,:,1]
yA = yA.reshape(-1)
yB = yB.reshape(-1)

In [ ]:
np.save("predictions/y_pred_1_"+t_id+".npy",y0)
np.save("predictions/y_true_1_"+t_id+".npy",yA)
np.save("predictions/y_pred_2_"+t_id+".npy",y1)
np.save("predictions/y_true_2_"+t_id+".npy",yB)